In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import StandardScaler
import torch
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
from ultralytics import YOLO


# Graffiti Heat Map Image Prediction

YOLOv11

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_masked_images_df(root_dir):
    data = []

    for subfolder_A in os.listdir(root_dir):
        path_A = os.path.join(root_dir, subfolder_A)
        if os.path.isdir(path_A):  # Ensure it's a directory
            for subfolder_B in os.listdir(path_A):
                path_B = os.path.join(path_A, subfolder_B)
                if os.path.isdir(path_B) and "_New" in subfolder_B:
                    # Extract key up to "_INTERIOR_Masked"
                    folder_key = subfolder_B.split('_Images_New')[0]

                    for filename in os.listdir(path_B):
                        file_path = os.path.join(path_B, filename)
                        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                            data.append((folder_key, file_path))

    # Create DataFrame
    df = pd.DataFrame(data, columns=["Neighborhood", "image_path"])
    return df


# Example usage
root_directory = "/content/drive/Shareddrives/UChicago Data for the Common Good/Dataset/All Chicago Neighborhoods"
masked_images_df = get_masked_images_df(root_directory)

# Display the DataFrame


In [ ]:
#Collecting a random sample of 250 images per neighborhood
masked_samples_GRAFFITI = masked_images_df.groupby('Neighborhood', group_keys=False).apply(lambda x: x.sample(n=min(len(x), 250), random_state=42))


<ipython-input-16-000b9b766295>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  masked_samples_GRAFFITI = masked_images_df.groupby('Neighborhood', group_keys=False).apply(lambda x: x.sample(n=min(len(x), 250), random_state=42))


In [ ]:
masked_samples_GRAFFITI.head()

,Neighborhood,image_path
5861,ALBANY_PARK_INTERIOR,/content/drive/Shareddrives/UChicago Data for ...
5573,ALBANY_PARK_INTERIOR,/content/drive/Shareddrives/UChicago Data for ...
5874,ALBANY_PARK_INTERIOR,/content/drive/Shareddrives/UChicago Data for ...
5655,ALBANY_PARK_INTERIOR,/content/drive/Shareddrives/UChicago Data for ...
5604,ALBANY_PARK_INTERIOR,/content/drive/Shareddrives/UChicago Data for ...


In [ ]:
model_weights_path_GRAFFITI = '/content/drive/Shareddrives/UChicago Data for the Common Good/Heatmap/Sample Heatmap/graffiti.pt'

model_data_GRAFFITI = torch.load(model_weights_path_GRAFFITI, map_location=torch.device('cuda'))  # Use 'cuda' if using GPU
print(type(model_data_GRAFFITI))


<class 'dict'>


In [ ]:
checkpoint = torch.load(model_weights_path_GRAFFITI, map_location=torch.device('cpu'))
print(checkpoint.keys())

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.we

In [ ]:
# Load the YOLOv11 model
model_weights_path_GRAFFITI = '/content/drive/Shareddrives/UChicago Data for the Common Good/Heatmap/Sample Heatmap/graffiti.pt'
# Initialize the YOLO model (Modify this based on your YOLOv11 model class)
model_GRAFFITI = YOLO(model_weights_path_GRAFFITI)  # Adjust this if your class has specific parameters

def predict(image_path):
  try:
    # Load an image for prediction
    image = Image.open(image_path)

    # Run inference
    results = model_GRAFFITI(image)

    # Confidence threshold
    conf_threshold = 0.3

    # Check if any detections meet the confidence threshold
    has_class = any(det.conf > conf_threshold for det in results[0].boxes)
    return has_class
  except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

masked_samples_GRAFFITI["Prediction"] = masked_samples_GRAFFITI["image_path"].apply(predict)


Streaming output truncated to the last 5000 lines.
0: 416x416 (no detections), 10.1ms
Speed: 1.6ms preprocess, 10.1ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 13.4ms
Speed: 1.9ms preprocess, 13.4ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 10.2ms
Speed: 1.6ms preprocess, 10.2ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 10.5ms
Speed: 1.6ms preprocess, 10.5ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 10.6ms
Speed: 1.7ms preprocess, 10.6ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 12.9ms
Speed: 1.8ms preprocess, 12.9ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 10.3ms
Speed: 1.6ms preprocess, 10.3ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no det

In [ ]:
masked_samples_GRAFFITI['Prediction'].value_counts()

,count
Prediction,
False,19207
True,43


# Bad Roads Heatmap Image Prediction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_weights_path = '/content/drive/Shareddrives/UChicago Data for the Common Good/BAD ROADS ONLY model results/best_model_binary_20250216_052705/best_model_binary.pth'

model_data = torch.load(model_weights_path, map_location=torch.device('cuda'))  # Use 'cuda' if using GPU
print(type(model_data))


<ipython-input-10-344e69215580>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_data = torch.load(model_weights_path, map_location=torch.device('cuda'))  # Use 'cuda'

<class 'collections.OrderedDict'>


In [ ]:
# Use a pretrained ResNet101, then modify the final layer for 1 output.
model = models.resnet101(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # 1 output, matching your checkpoint

model_path = '/content/drive/Shareddrives/UChicago Data for the Common Good/BAD ROADS ONLY model results/best_model_binary_20250216_052705/best_model_binary.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))
model = model.to('cuda')
model.eval()

print("Model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-23-54c91fe4fef0>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

Model loaded successfully!


# Getting the Images

In [ ]:


def get_masked_images_df(root_dir):
    data = []

    for subfolder_A in os.listdir(root_dir):
        path_A = os.path.join(root_dir, subfolder_A)
        if os.path.isdir(path_A):  # Ensure it's a directory
            for subfolder_B in os.listdir(path_A):
                path_B = os.path.join(path_A, subfolder_B)
                if os.path.isdir(path_B) and "Masked" in subfolder_B:
                    # Extract key up to "_INTERIOR_Masked"
                    folder_key = subfolder_B.split('_INTERIOR_Masked')[0]

                    for filename in os.listdir(path_B):
                        file_path = os.path.join(path_B, filename)
                        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                            data.append((folder_key, file_path))

    # Create DataFrame
    df = pd.DataFrame(data, columns=["Neighborhood", "image_path"])
    return df


# Example usage
root_directory = "/content/drive/Shareddrives/UChicago Data for the Common Good/Dataset/All Chicago Neighborhoods (Missing)"
masked_images_df = get_masked_images_df(root_directory)

# Display the DataFrame


In [ ]:
#masked_images_df['Neighborhood'] = masked_images_df['Neighborhood'].str.lower()
masked_samples = masked_images_df.groupby('Neighborhood', group_keys=False).apply(lambda x: x.sample(n=min(len(x), 250), random_state=42))


<ipython-input-32-ba6195d4cf89>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  masked_samples = masked_images_df.groupby('Neighborhood', group_keys=False).apply(lambda x: x.sample(n=min(len(x), 250), random_state=42))


In [ ]:
masked_samples.head()

,Neighborhood,image_path
12897,ANDERSONVILLE,/content/drive/Shareddrives/UChicago Data for ...
12609,ANDERSONVILLE,/content/drive/Shareddrives/UChicago Data for ...
12910,ANDERSONVILLE,/content/drive/Shareddrives/UChicago Data for ...
12691,ANDERSONVILLE,/content/drive/Shareddrives/UChicago Data for ...
12640,ANDERSONVILLE,/content/drive/Shareddrives/UChicago Data for ...


In [ ]:
transform = transforms.Compose([
    transforms.Resize((640, 640)),  # Resize if needed
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ResNet normalization

])

def predict(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0)  # Add batch dimension
        image = image.to(torch.device('cuda'))  # Move to GPU if available
        with torch.no_grad():
            output = model(image)
        prediction = torch.sigmoid(output).item()  # Convert to probability if needed
        return 1 if prediction > 0.5 else 0  # Threshold decision
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# Apply prediction function to image_path column
# temp_df = masked_images_df.head()
masked_samples["Prediction"] = masked_samples["image_path"].apply(predict)

#masked_images_df["Prediction"] = masked_images_df["image_path"].apply(predict)

# # Save the updated DataFrame
# df.to_csv("updated_predictions.csv", index=False)


In [ ]:
masked_samples_groupd = masked_samples.groupby('Neighborhood')['Prediction'].sum().reset_index()
masked_samples_groupd['Neighborhood'] = masked_samples_groupd['Neighborhood'].str.lower()

In [ ]:
masked_samples_groupd.to_csv('Missing_Neighborhoods.csv')